In [9]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("semaphore-a7fd1-firebase-adminsdk-dyeb3-e430d9430a.json")


In [10]:
app = firebase_admin.initialize_app(cred)

db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [16]:
doc_ref = db.collection(u'posts')
docs = doc_ref.stream()
posts = []
comments = []
likes = []

for doc in docs:
    comments_ref = db.collection(u'posts').document(doc.id).collection('comments')
    
    for comment in comments_ref.stream():
        comments.append(comment.to_dict())
        
    posts.append(doc.to_dict())


[{'comment': 'clear out all the garbage apps from it', 'userImg': 'https://firebasestorage.googleapis.com/v0/b/semaphore-a7fd1.appspot.com/o/users%2Fj3yzggbxE9FCb4QTIHZR%2Fimage?alt=media&token=16b8c731-8d94-490f-bcd0-1ae771593f46', 'id': 'j3yzggbxE9FCb4QTIHZR', 'timestamp': DatetimeWithNanoseconds(2023, 1, 6, 10, 52, 23, 534000, tzinfo=datetime.timezone.utc), 'username': 'Rasp Neri', 'tag': 'raspneri'}, {'comment': 'keep testing', 'tag': 'raspneri', 'id': 'j3yzggbxE9FCb4QTIHZR', 'timestamp': DatetimeWithNanoseconds(2023, 1, 5, 16, 12, 0, 788000, tzinfo=datetime.timezone.utc), 'username': 'Rasp Neri', 'userImg': 'https://firebasestorage.googleapis.com/v0/b/semaphore-a7fd1.appspot.com/o/users%2Fj3yzggbxE9FCb4QTIHZR%2Fimage?alt=media&token=16b8c731-8d94-490f-bcd0-1ae771593f46'}, {'comment': "how's the testing going?", 'userImg': 'https://lh3.googleusercontent.com/a/AEdFTp6sqFzAnm4SBmuzlJL2rulbgl1NZHSEpMQIMwXQ=s96-c', 'id': 'CYIMUn78OTWEsb9iDhWS', 'timestamp': DatetimeWithNanoseconds(2023

In [4]:
import pandas as pd

In [5]:
posts_df = pd.DataFrame(posts)

In [12]:
posts_df.head()

,text,tag,id,timestamp,username,userImg
0,My phone has slowed down so much...,zackzomor,CYIMUn78OTWEsb9iDhWS,2023-01-05 16:03:55.595000+00:00,Zack Zomor,https://lh3.googleusercontent.com/a/AEdFTp6sqF...
1,"Gruppa Khmuri is a really good band, would rea...",raspneri,j3yzggbxE9FCb4QTIHZR,2023-01-05 16:13:45.643000+00:00,Rasp Neri,https://firebasestorage.googleapis.com/v0/b/se...
2,"Lorem ipsum dolor sit amet, consectetur adipis...",jessedixon,gjNJo321L4j7LRA79xYD,2022-12-26 16:19:48.032000+00:00,Jesse Dixon,https://lh3.googleusercontent.com/a/AEdFTp6ezs...
3,Biden is not that great actually,zackzomor,CYIMUn78OTWEsb9iDhWS,2023-01-05 16:14:46.499000+00:00,Zack Zomor,https://lh3.googleusercontent.com/a/AEdFTp6sqF...
4,Test,jessedixon,gjNJo321L4j7LRA79xYD,2022-12-26 17:47:47.769000+00:00,Jesse Dixon,https://lh3.googleusercontent.com/a/AEdFTp6ezs...


In [29]:
posts_df_grouped = posts_df.groupby("username")["text"].count()
max_user = posts_df_grouped.idxmax()
max_amount = posts_df_grouped.max()
print(f"User:{max_user}, Posts:{max_amount}")

User:Jesse Dixon, Posts:5


In [28]:
posts_df_grouped

'Jesse Dixon'

In [18]:
comments_df = pd.DataFrame(comments)
comments_df_grouped = comments_df.groupby("username")["comment"].count()
max_user_comments = comments_df_grouped.idxmax()
max_amount_comments = comments_df_grouped.max()

print(f"User:{max_user_comments}, Comments:{max_amount_comments}")

User:Rasp Neri, Comments:2


In [21]:
import pandas as pd
import numpy as np
import requests
import nltk
import string
import re
import os
from collections import Counter

In [22]:
posts = posts_df['text']

In [49]:
stop_words = ["a", "an","I","i", "the", "this", "that", "it", "is", "really", "to", "and", "for", "these", "those"] + list(string.punctuation)

In [43]:
filtered_wordlists = []
for post in posts:
    wordlist = [word for word in nltk.word_tokenize(post) if word not in stop_words] #a list of words per tweet
    filtered_wordlists.append(wordlist)
filtered_wordlists[0]

['My', 'phone', 'has', 'slowed', 'down', 'so', 'much', '...']

In [44]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/inferno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
filtered_wordlists

[['My', 'phone', 'has', 'slowed', 'down', 'so', 'much', '...'],
 ['Gruppa', 'Khmuri', 'good', 'band', 'would', 'recommended'],
 ['Lorem',
  'ipsum',
  'dolor',
  'sit',
  'amet',
  'consectetur',
  'adipiscing',
  'elit',
  'Vestibulum',
  'aliquam',
  'elementum',
  'massa',
  'sollicitudin',
  'Praesent',
  '😀🙃🙃'],
 ['Biden', 'not', 'great', 'actually'],
 ['Test'],
 ['in',
  'end',
  'I',
  "'m",
  'thankful',
  'we',
  'have',
  'Biden',
  'instead',
  'of',
  'other',
  'guy'],
 ['The', 'most', 'dangerous', 'thing', 'in', 'world', 'bad', 'decision'],
 ['Biden', "n't", 'bad', 'actually'],
 ['Eating', 'healthy', 'important'],
 ['Let', "'s", 'try', 'app', 'out'],
 ['Lorem',
  'ipsum',
  'dolor',
  'sit',
  'amet',
  'consectetur',
  'adipiscing',
  'elit',
  'Nunc',
  'scelerisque',
  'sed',
  'nunc',
  'ac',
  'laoreet',
  'Class',
  '😄😄😄'],
 ['Nice', 'i', 'can', 'write', 'hare'],
 ['I',
  'ate',
  '6',
  'donuts',
  'breakfast',
  '4',
  'lunch',
  '3',
  'dinner',
  '1',
  'dessert

In [46]:
all_words = [item
              for sublist in filtered_wordlists
              for item in sublist]

In [47]:
all_words

['My',
 'phone',
 'has',
 'slowed',
 'down',
 'so',
 'much',
 '...',
 'Gruppa',
 'Khmuri',
 'good',
 'band',
 'would',
 'recommended',
 'Lorem',
 'ipsum',
 'dolor',
 'sit',
 'amet',
 'consectetur',
 'adipiscing',
 'elit',
 'Vestibulum',
 'aliquam',
 'elementum',
 'massa',
 'sollicitudin',
 'Praesent',
 '😀🙃🙃',
 'Biden',
 'not',
 'great',
 'actually',
 'Test',
 'in',
 'end',
 'I',
 "'m",
 'thankful',
 'we',
 'have',
 'Biden',
 'instead',
 'of',
 'other',
 'guy',
 'The',
 'most',
 'dangerous',
 'thing',
 'in',
 'world',
 'bad',
 'decision',
 'Biden',
 "n't",
 'bad',
 'actually',
 'Eating',
 'healthy',
 'important',
 'Let',
 "'s",
 'try',
 'app',
 'out',
 'Lorem',
 'ipsum',
 'dolor',
 'sit',
 'amet',
 'consectetur',
 'adipiscing',
 'elit',
 'Nunc',
 'scelerisque',
 'sed',
 'nunc',
 'ac',
 'laoreet',
 'Class',
 '😄😄😄',
 'Nice',
 'i',
 'can',
 'write',
 'hare',
 'I',
 'ate',
 '6',
 'donuts',
 'breakfast',
 '4',
 'lunch',
 '3',
 'dinner',
 '1',
 'dessert',
 'Do',
 'I',
 'have',
 'diabetes',
 '

In [48]:
word_counter = Counter(all_words)
most_common_words = word_counter.most_common(10)
most_common_words

[('I', 4),
 ('Biden', 3),
 ('Test', 3),
 ('Lorem', 2),
 ('ipsum', 2),
 ('dolor', 2),
 ('sit', 2),
 ('amet', 2),
 ('consectetur', 2),
 ('adipiscing', 2)]